# Code to augment the SQuAD dataset with CoQA

In [ ]:
! pip install git+https://github.com/huggingface/datasets.git

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-772fd2bc
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-772fd2bc
     |████████████████████████████████| 212 kB 15.7 MB/s 
     |████████████████████████████████| 136 kB 32.7 MB/s 
     |████████████████████████████████| 1.1 MB 54.5 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 127 kB 73.9 MB/s 
     |████████████████████████████████| 144 kB 60.7 MB/s 
     |████████████████████████████████| 271 kB 54.0 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Created wheel for datasets: filename=datasets-2.1.1.dev0-py3-none-any.whl size=333947 sha256=c80d39bae16b0463394cf2058866d3d6609f86d92a1cc9eda08257878ba5ca00
  Stored in directory: /tmp/pip-ephem-wheel-cache-lqm_0bnq/wheels/cc/33/c1/2cacc415b23189a83908e45db67381ba26175ef1e8aa9062aa
Successfully built datasets
  Attempting uninstall: urllib3
    Fo

In [ ]:
from datasets import load_dataset, load_metric
import datasets

## Download both the datasets

In [ ]:
dataset_squad = load_dataset("squad_v2")
dataset_coqa = load_dataset("coqa")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7199 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset coqa downloaded and prepared to /root/.cache/huggingface/datasets/coqa/default/1.0.0/553ce70bfdcd15ff4b5f4abc4fc2f37137139cde1f58f4f60384a53a327716f0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset_squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
dataset_coqa

DatasetDict({
    train: Dataset({
        features: ['source', 'story', 'questions', 'answers'],
        num_rows: 7199
    })
    validation: Dataset({
        features: ['source', 'story', 'questions', 'answers'],
        num_rows: 500
    })
})

##

## Bring both the datasets into the same format
Since both datasets have different column names we change CoQA into SQuAD format

In [ ]:
coqa_dataset_train = []

In [ ]:
id = 0
for example in dataset_coqa['train']:
  answer_starts = example['answers']['answer_start']
  answer_texts = example['answers']['input_text']
  questions = example['questions']
  context = example['story']
  title = example['source']

  for i in range(len(answer_starts)):
    answers = {'answer_start' : [answer_starts[i]] , 'text' : [answer_texts[i]]}
    question = questions[i]
    temp = {'id' : str(id) , 'answers' : answers, 'question' : question , 'context' : context , 'title' : title}
    coqa_dataset_train.append(temp)
    id += 1


In [ ]:
coqa_dataset_val = []

In [ ]:
id = 0
for example in dataset_coqa['validation']:
  answer_starts = example['answers']['answer_start']
  answer_texts = example['answers']['input_text']
  questions = example['questions']
  context = example['story']
  title = example['source']

  for i in range(len(answer_starts)):
    answers = {'answer_start' : [answer_starts[i]] , 'text' : [answer_texts[i]] }
    question = questions[i]
    temp = {'id' : str(id) , 'answers' : answers, 'question' : question , 'context' : context , 'title' : title}
    coqa_dataset_val.append(temp)
    id += 1


In [ ]:
len(coqa_dataset_train)

108647

In [ ]:
len(coqa_dataset_val)

7983

In [ ]:
import pandas as pd

In [ ]:
coqa_train_df = pd.DataFrame(coqa_dataset_train)
coqa_val_df = pd.DataFrame(coqa_dataset_val)

In [ ]:
squad_train_df = pd.DataFrame(dataset_squad['train'])
squad_test_df = pd.DataFrame(dataset_squad['validation'])

## Join the two dataframes for the two datasets

In [ ]:
joined_dataset_train = pd.concat([squad_train_df , coqa_train_df])
joined_dataset_test = pd.concat([squad_test_df , coqa_val_df])

In [ ]:
joined_dataset_train.head()

,id,title,context,question,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start'..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"{'text': ['singing and dancing'], 'answer_star..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"{'text': ['2003'], 'answer_start': [526]}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"{'text': ['Houston, Texas'], 'answer_start': [..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"{'text': ['late 1990s'], 'answer_start': [276]}"


In [ ]:
joined_train = datasets.Dataset.from_pandas(joined_dataset_train)
joined_test = datasets.Dataset.from_pandas(joined_dataset_test)

In [ ]:
final_dataset = datasets.DatasetDict({'train' : joined_train , 'validation' : joined_test})

In [ ]:
import pickle as pkl

## Save for reference

In [ ]:
with open('/content/drive/MyDrive/Spring22/CS769/Project/FInal/SQUAD_COQA.pkl', 'wb') as fp:
  pkl.dump(final_dataset, fp)

## Creation of Uniform Dataset
We set a threshold of 10,000 samples per each type of question for the train dataset. This number can be varied depending on the dataset

In [ ]:
import pickle as pkl
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/Spring22/CS769/Project/FInal/SQUAD_COQA.pkl', 'rb') as fp:
  dataset = pkl.load(fp)

In [ ]:
final_df_train = pd.DataFrame(dataset['train'])

In [ ]:
qa_dict = {'what':[] , 'where': [], 'how': [], 'why':[], 'when': [], 'which':[],  'misc': [], 'who' : []}
qa_keys = ['what', 'where', 'how', 'why', 'when', 'which', 'who']

In [ ]:
for i in range(len(final_df_train)):
  question = final_df_train.iloc[i]['question']
  id = final_df_train.iloc[i]['id']
  misc_flag = True
  for key in qa_keys:
    if key in question.lower():
      qa_dict[key].append(id)
      misc_flag = False
      break
  if misc_flag == True:
    qa_dict['misc'].append(id)

In [ ]:
print({key : len(qa_dict[key]) for key in qa_dict.keys()})

{'what': 114236, 'where': 13231, 'how': 25438, 'why': 4881, 'when': 12938, 'which': 10090, 'misc': 26943, 'who': 31209}


In [ ]:
sorted({key : len(qa_dict[key]) for key in qa_dict.keys()}.items(), key = lambda x : x[1] , reverse = True)

[('what', 114236),
 ('who', 31209),
 ('misc', 26943),
 ('how', 25438),
 ('where', 13231),
 ('when', 12938),
 ('which', 10090),
 ('why', 4881)]

In [ ]:
import numpy as np

In [ ]:
major_key = ['what', 'where', 'how', 'when', 'which', 'who', 'misc']
for key in qa_dict:
  if key in major_key:
    np.random.shuffle(qa_dict[key])
    qa_dict[key] = qa_dict[key][:10000]

In [ ]:
sorted({key : len(qa_dict[key]) for key in qa_dict.keys()}.items(), key = lambda x : x[1] , reverse = True)

[('what', 10000),
 ('where', 10000),
 ('how', 10000),
 ('when', 10000),
 ('which', 10000),
 ('misc', 10000),
 ('who', 10000),
 ('why', 4881)]

## Select from the dataframe only those ids in qa_dict

In [ ]:
values_list = []
for key in qa_dict:
  for value in qa_dict[key]:
    values_list.append(value)

In [ ]:
len(values_list)

74881

In [ ]:
expt = final_df_train['id'].isin(values_list)

In [ ]:
expt

0          True
1         False
2          True
3         False
4          True
          ...  
238961     True
238962     True
238963    False
238964    False
238965    False
Name: id, Length: 238966, dtype: bool

In [ ]:
final_df_train['temp'] = expt

In [ ]:
final_df_train = final_df_train[final_df_train['temp'] == True]

In [ ]:
len(final_df_train)

74881

In [ ]:
final_df_train = final_df_train.drop('temp' , axis = 1)

In [ ]:
final_df_train = final_df_train.drop('__index_level_0__' , axis = 1)

In [ ]:
import copy

## Create a constant test set for validation of all methods

In [ ]:
constant_test_df = pd.DataFrame(dataset_squad['validation'])

In [ ]:
len(constant_test_df)

11873

In [ ]:
uniform_train = datasets.Dataset.from_pandas(final_df_train)
constant_test = datasets.Dataset.from_pandas(constant_test_df)

In [ ]:
uniform_train

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', '__index_level_0__'],
    num_rows: 74881
})

In [ ]:
uniform_dataset = datasets.DatasetDict({'train' : uniform_train , 'test' : constant_test})

In [ ]:
uniform_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', '__index_level_0__'],
        num_rows: 74881
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
with open('/content/drive/MyDrive/Spring22/CS769/Project/FInal/SQUAD_COQA_uniform.pkl' , 'wb') as fp:
  pkl.dump(uniform_dataset, fp)